In [7]:
from agents import Agent, Runner, SQLiteSession, function_tool, trace
from pydantic import BaseModel

session = SQLiteSession(session_id="user_5", db_path="ai-memory.db")


class Answer(BaseModel):
    answer: str
    reason: str


@function_tool
def get_weather(city: str) -> str:
    return "30 degrees"


geography_agent = Agent(
    name="Geography Agent",
    model="gpt-5-nano-2025-08-07",
    instructions="You are a geography agent. Answer the user's question about geography.",
    handoff_description="You are a geography agent. Answer the user's question about geography.",
    tools=[get_weather],
    output_type=Answer,
)

economics_agent = Agent(
    name="Economics Agent",
    model="gpt-5-nano-2025-08-07",
    instructions="You are a economics agent. Answer the user's question about economics.",
    handoff_description="You are a economics agent. Answer the user's question about economics.",
    output_type=Answer,
)


main_agent = Agent(
    name="Assistant Agent",
    model="gpt-5-nano-2025-08-07",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[geography_agent, economics_agent],
    output_type=Answer,
)


In [8]:
with trace("user_5") as t:
    result = await Runner.run(
        main_agent,
        "What is the capital of France?",
        session=session,
    )
    result = await Runner.run(
        main_agent,
        "What is the capital of Thailand?",
        session=session,
    )
    result = await Runner.run(
        main_agent,
        "What is the capital of Brazil?",
        session=session,
    )

    print(result.last_agent.name)
    print(result.final_output)

Geography Agent
answer='Brasília' reason='Question handled by Geography Agent. The capital of Brazil is Brasília.'
